# Road lane detector 

## Detect lane class

In [1]:
import matplotlib.pylab as plt
import cv2
import numpy as np

class RoadLaneDetector:
    """
    The RoadLaneDetector contains function to detect lanes
    """
    def region_of_interest(self, img, vertices):
        """
        Extracting a specific region in the photo where the line detection algorithm is to work
        
        Args:
            img (np.array): One frame of the video
            vertices (list): Vertex list to define the region to extract
            
        Returns:
            Result. This is a scalar if both x1 and x2 are scalars.
            
        """
        
        mask = np.zeros_like(img)
        match_mask_color = 255
        cv2.fillPoly(mask, vertices, match_mask_color)
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image

    def drow_the_lines(self, img, lines):
        """
        Drow the road lines
        
        Args:
            img (np.array): One frame of the video
            lines (cv2.HoughLinesP): Hough Line Transform algorithm
            
        Returns:
            Result. Image with lines
        """
        
        img = np.copy(img)
        blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=10)

        img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0)
        return img

    def process(self, image, thresh, minLine, maxLine):
        """
        The process of detecting lines on the road
        
        Args:
            image (np.array): One frame of the video
            thresh (int): The minimum number of intersecting points to detect a line
            minLine (int): Minimum line length. Line segments shorter than that are rejected.
            maxLine (int): Maximum allowed gap between points on the same line to link them.
            
        Returns:
            Result. Image with lines (drow_the_lines method)
        """
        height = image.shape[0]
        width = image.shape[1]
        region_of_interest_vertices = [
            [0, height],
            [width/2, height/2],
            [width, height]
        ]
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        canny_image = cv2.Canny(gray_image, 100, 100)
        cropped_image = self.region_of_interest(canny_image,
                        np.array([region_of_interest_vertices], np.int32),)

        lines = cv2.HoughLinesP(cropped_image,
                                rho=1,
                                theta=np.pi/180,
                                threshold=thresh,
                                lines=np.array([]),
                                minLineLength=minLine,
                                maxLineGap=maxLine)
        image_with_lines = self.drow_the_lines(image, lines)
        return image_with_lines

## GUI class

In [2]:
from PyQt5 import QtWidgets, uic
import sys

class Ui(QtWidgets.QMainWindow):
    """
    Graphical User Interface
    """
    def __init__(self):
        super(Ui, self).__init__()
        uic.loadUi('GUI.ui', self)

        self.filePath = self.findChild(QtWidgets.QTextEdit, 'filePath_Text')
        
        self.threshold = self.findChild(QtWidgets.QSlider, 'threshold_Slider') 
        self.minLineLength = self.findChild(QtWidgets.QSlider, 'minLineLength_Slider') 
        self.maxLineGap = self.findChild(QtWidgets.QSlider, 'maxLineGap_Slider') 
        
        self.threshold_Value = self.findChild(QtWidgets.QLabel, 'threshold_Value')
        self.threshold.valueChanged.connect(self.change_thresholdValue)
        
        self.minLineLength_Value = self.findChild(QtWidgets.QLabel, 'minLineLength_Value')
        self.minLineLength.valueChanged.connect(self.change_minLineLengthValue)
        
        self.maxLineGap_Value = self.findChild(QtWidgets.QLabel, 'maxLineGap_Value')
        self.maxLineGap.valueChanged.connect(self.change_maxLineGapValue)
        
        self.button = self.findChild(QtWidgets.QPushButton, 'pushButton')
        self.button.clicked.connect(self.sendInformations) 
        
        self.show()

    def sendInformations(self):
        """
        Pushing button event handler.
        """
        RLD = RoadLaneDetector()
        cap = cv2.VideoCapture(self.filePath.toPlainText())

        while cap.isOpened():
            ret, frame = cap.read()
            frame = RLD.process(frame, 
                                     self.threshold.value(),
                                     self.minLineLength.value(),
                                     self.maxLineGap.value())
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
    
    def change_thresholdValue(self):
        """
        Show value from Slider. 
        """
        self.threshold_Value.setText(str(self.threshold.value()))
        
    def change_minLineLengthValue(self):
        """
        Show value from Slider. 
        """
        self.minLineLength_Value.setText(str(self.minLineLength.value()))
        
    def change_maxLineGapValue(self):
        """
        Show value from Slider. 
        """
        self.maxLineGap_Value.setText(str(self.maxLineGap.value()))

## Main program

In [3]:
if __name__ == "__main__": 
    app = QtWidgets.QApplication(sys.argv)
    window = Ui()
    app.exec_()
